In [ ]:
import sys
import torch
from torch import nn
import os
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1
EPOCHS = 10

In [ ]:
from csv_data import WeatherDatasetWrapperRNN

class WeatherDataset(WeatherDatasetWrapperRNN):
    predictors = ["tmax", "tmin", "rain"]
    target = "tmax_tomorrow"
    sequence_length = 7

wrapper = WeatherDataset()

# This is multiple lines
datasets = wrapper.generate_datasets(BATCH_SIZE)
train = datasets["train"]
test = datasets["test"]
valid = datasets["validation"]

In [ ]:
def mse(actual, predicted):
    return torch.mean((actual-predicted)**2)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        torch.manual_seed(0)
        self.rnn = nn.RNN(3, 4, 1, batch_first=True)
        self.dense = nn.Linear(4, 1)

    def forward(self, x):
        x, hidden = self.rnn(x)
        x = self.dense(x)
        return x

In [ ]:
model = NeuralNetwork().to(DEVICE)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for epoch in range(EPOCHS):
    epoch_loss = 0
    for batch, (sequence, target) in enumerate(train):
        optimizer.zero_grad()

        sequence = sequence.to(DEVICE)
        pred = model(sequence)

        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()

        epoch_loss += mse(target, pred)

    losses = []
    with torch.no_grad():
        valid_loss = 0
        for batch, (sequence, target) in enumerate(valid):
            sequence = sequence.to(DEVICE)
            pred = model(sequence)
            loss = loss_fn(pred, target)
            valid_loss += mse(target, pred)

    print(f"Epoch {epoch}\tTrain Loss: {epoch_loss / len(train)}\tValid Loss: {valid_loss / len(valid)}")

In [ ]:
[p for p in model.named_parameters()]